In [2]:
import sys
sys.path.append('/home/francisco/hons/HonsProOpen/MLPHonoursExtension/mlp')
sys.path.append('/home/francisco/hons/HonsProOpen/MLPHonoursExtension')
sys.path.append('..')

In [3]:
from copy import deepcopy
from mlp.layers import MLP, Linear, Sigmoid, Softmax #import required layer types
from mlp.layers import * 
from mlp.optimisers import SGDOptimiser #import the optimiser

from mlp.costs import CECost, MSECost #import the cost we want to use for optimisation
from mlp.schedulers import LearningRateFixed
import numpy
import logging
from mlp.dataset import *
from base import *

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MACLDataProvider(dset='train', batch_size=100,
                            max_num_batches=-10, randomize=True,name='RLAx', conv_reshape=False)
valid_dp = MACLDataProvider(dset='valid', batch_size=1140,
                            max_num_batches=1, randomize=False,name='RLAx', conv_reshape=False)


test_dp = MACLDataProvider(dset='test', batch_size=1140,
                            max_num_batches=1, randomize=False,name='RLAx', conv_reshape=False)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
rng = numpy.random.RandomState([2015,10,10])

INFO:root:Initialising data providers...


(6840, 45, 125) (6840,)
(6840, 125)
(1140, 45, 125) (1140,)
(1140, 125)
(1140, 45, 125) (1140,)
(1140, 125)


In [1]:
#some hyper-parameters
nhid = 100
learning_rate =0.05
max_epochs = 1000

cost = CECost()    
stats = list()

# test_dp = deepcopy(valid_dp)
train_dp.reset()
valid_dp.reset()
test_dp.reset()

#define the model
model = MLP(cost=cost)
model.add_layer(ComplexAbs(idim=125, odim=125, irange=1.6, rng=rng))
# Every activation function from dft layer produces two values (x,y) for x+iy
#model.add_layer(Sigmoid(idim=125, odim=125, irange=1.6, rng=rng))
model.add_layer(Sigmoid(idim=125, odim=125, irange=1.6, rng=rng))

model.add_layer(Softmax(idim=125, odim=19, rng=rng))

# define the optimiser, here stochasitc gradient descent
# with fixed learning rate and max_epochs
lr_scheduler = LearningRateFixed(learning_rate=learning_rate, max_epochs=max_epochs)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)

logger.info('Training started...')
tr_stats_f, valid_stats_f = optimiser.train(model, train_dp, test_dp)

logger.info('Testing the model on test set:')

tst_costf, tst_accuracyf = optimiser.validate(model,valid_dp )
logger.info('ACL test set accuracy is %.2f %%, cost (%s) is %.3f'%
            (tst_accuracyf*100., cost.get_name(), tst_costf))

NameError: name 'CECost' is not defined

In [15]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logger.info('Initialising data providers...')

train_dp = MACLDataProvider(dset='train', batch_size=100,
                            max_num_batches=-10, randomize=True,name='RLAx', conv_reshape=True)
valid_dp = MACLDataProvider(dset='valid', batch_size=1140,
                            max_num_batches=1, randomize=False,name='RLAx', conv_reshape=True)


test_dp = MACLDataProvider(dset='test', batch_size=1140,
                            max_num_batches=1, randomize=False,name='RLAx', conv_reshape=True)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

INFO:root:Initialising data providers...


(6840, 45, 125) (6840,)
(6840, 125)
(1140, 45, 125) (1140,)
(1140, 125)
(1140, 45, 125) (1140,)
(1140, 125)


In [17]:
# reload("mlp/con")
from mlp.convlin import *
rng = numpy.random.RandomState([2015,10,10])

cost = CECost()
model = MLP(cost=cost)
model.add_layer(ComplexAbs(idim=125, odim=125, irange=1.6, rng=rng))
model.add_layer(ConvRelu_Opt(1, 5, rng=rng, stride=(1,1)))
#model.add_layer(ConvRelu_Opt(5, 5, rng=rng, stride=(1,1), image_shape=(122,)))
# model.add_layer(ConvMaxPool2D(5, (100,5, 24,24)))
# model.add_layer(ConvLinear_Opt(5, 10, rng=rng, image_shape=(12, 12) , kernel_shape=(5,5)))
# model.add_layer(ConvMaxPool2D(10, (100,10, 8,8), pool_shape=(2,2)))
# sigmoid 
model.add_layer(Sigmoid(idim=122*5, odim=122*5, rng=rng))

model.add_layer(Softmax(idim=122*5, odim=19, rng=rng))
# for sigmoid 0.1 is good for linear 0.005 and 0.1 is so so bigger not so good. ReLu : 0.1
lr_scheduler = LearningRateFixed(learning_rate=0.01, max_epochs=1000)
optimiser = SGDOptimiser(lr_scheduler=lr_scheduler)
val, tra = optimiser.train(model, train_dp, test_dp)

tst_costf, tst_accuracyf = optimiser.validate(model,valid_dp )
logger.info('ACL test set accuracy is %.2f %%, cost (%s) is %.3f'%
            (tst_accuracyf*100., cost.get_name(), tst_costf))

INFO:mlp.optimisers:Epoch 0: Training cost (ce) for initial model is 3.260. Accuracy is 13.03%
INFO:mlp.optimisers:Epoch 0: Validation cost (ce) for initial model is 3.220. Accuracy is 9.47%
INFO:mlp.optimisers:Epoch 1: Training cost (ce) is 2.120. Accuracy is 45.46%
INFO:mlp.optimisers:Epoch 1: Validation cost (ce) is 1.772. Accuracy is 48.33%
INFO:mlp.optimisers:Epoch 1: Took 3 seconds. Training speed 2086 pps. Validation speed 6712 pps.
INFO:mlp.optimisers:Epoch 2: Training cost (ce) is 1.443. Accuracy is 66.57%
INFO:mlp.optimisers:Epoch 2: Validation cost (ce) is 1.433. Accuracy is 56.58%
INFO:mlp.optimisers:Epoch 2: Took 3 seconds. Training speed 2139 pps. Validation speed 7131 pps.
INFO:mlp.optimisers:Epoch 3: Training cost (ce) is 1.208. Accuracy is 71.76%
INFO:mlp.optimisers:Epoch 3: Validation cost (ce) is 1.256. Accuracy is 68.25%
INFO:mlp.optimisers:Epoch 3: Took 3 seconds. Training speed 2201 pps. Validation speed 7131 pps.
INFO:mlp.optimisers:Epoch 4: Training cost (ce) is

In [23]:
valid_dp.reset()

tst_costf, tst_accuracyf = optimiser.validate(model,valid_dp )

In [25]:
tst_costf


0.73423245617895283

In [26]:
tst_accuracyf

0.77894736842105261